##### v4：tool function 可行版本

In [22]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
os.environ["GROQ_API_KEY"] = 'gsk_'
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],

    model_name="llama-3.3-70b-versatile",       

    temperature=0.0,
  #  max_tokens=1000,
)

In [ ]:
import re
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
import requests
from bs4 import BeautifulSoup  # 用於解析 HTML 格式

last_sparql_result = None

# === SPARQL Query Tool Function ===
def preprocess_sparql(query: str) -> str:
    # Remove unsupported SERVICE clause
    query = re.sub(r'SERVICE\s+wikibase:label\s*\{[^}]*\}', '', query, flags=re.IGNORECASE)
    # Add rdfs:label filter if not present
    if 'rdfs:label' not in query:
        query += '\nOPTIONAL { ?s rdfs:label ?label . FILTER(lang(?label) = "zh") }'
    # 基本格式檢查
    if 'SELECT' not in query.upper():
        query = f"SELECT * WHERE {{ {query} }}"
    return query.strip()

@tool("query_kg_tool", return_direct=True)
def query_kg_tool(sparql_query: str) -> str:
    """
    查詢本地 Virtuoso SPARQL endpoint，並回傳 JSON 格式的查詢結果摘要。
    輸入為 SPARQL 語句，會自動處理不支援語法與中文標籤補強。
    """
    global last_sparql_query
    print("大型語言產生的 SPARQL 查詢語句：\n", sparql_query)
    #cleaned_query = preprocess_sparql(sparql_query)
    #print("✅ 預處理後的 SPARQL 查詢語句：\n", cleaned_query)
    last_sparql_query = sparql_query
    
    endpoint = "http://192.168.133.39:8890/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    params = {"query": sparql_query}
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()

        # 如果返回的是 HTML 格式，使用 BeautifulSoup 解析
        if "html" in response.headers["Content-Type"]:
            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.find_all('tr')[1:]  # 排除表頭
            output = []
            for row in rows:
                cols = row.find_all('td')
                subject = cols[0].get_text()  # 主體
                predicate = cols[1].get_text()  # 關係
                object_ = cols[2].get_text()  # 客體
                output.append(f"Subject: {subject}, Predicate: {predicate}, Object: {object_}")
            return "\n".join(output)

        # 如果返回的是 JSON 格式
        results = response.json()
        bindings = results.get("results", {}).get("bindings", [])
        if not bindings:
            return "查無資料，請確認查詢條件或資料庫內容。"

        output = []
        for row in bindings:
            row_data = {k: v.get("value") for k, v in row.items()}
            output.append(str(row_data))

        #return "\n".join([f" {row}" for i, row in enumerate(output)])
        return "\n".join(output)
    except Exception as e:
        return f"查詢失敗，請確認 SPARQL 是否正確。錯誤：{str(e)}"

# === Tools List ===
tools = [
    query_kg_tool
]

# === Prompt Setup ===
system_prompt = """
你是一個智慧型知識查詢助理，所有問題都必須透過 query_kg_tool 工具查詢 RDF 知識圖譜，不能自己回答。
請將自然語言問題轉換成正確的 SPARQL 查詢後，使用 query_kg_tool 工具查詢。
請注意：
- 使用的 SPARQL endpoint 是本地 Virtuoso，不支援 Wikidata 特有語法如 SERVICE wikibase:label。
- 請使用標準 SPARQL 1.1 語法。
- 若要取得中文名稱，請使用 rdfs:label 並加入 FILTER(lang(?label) = \"zh\")。
"""

# === Initialize Agent ===
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "system_message": SystemMessage(content=system_prompt)
    }
)


In [24]:
# === Agent 使用示範 ===
question = "查詢與電影有關的文件"
response = agent.run(question)
if not response:
    print("⚠️ 無查詢結果，請檢查 SPARQL 或語句。")
else:
    print("🔎 查詢回覆：", response)

#print("SPARQL 查詢語句：", last_sparql_query)



> Entering new AgentExecutor chain...
Thought: 我需要查詢與電影有關的文件，所以我應該使用SPARQL語言來查詢相關的資料。
Action: query_kg_tool
Action Input: PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s rdf:type <http://example.org/Movie> . ?s ?p ?o . }大型語言產生的 SPARQL 查詢語句：
 PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s rdf:type <http://example.org/Movie> . ?s ?p ?o . }

Observation: 查無資料，請確認查詢條件或資料庫內容。


> Finished chain.
🔎 查詢回覆： 查無資料，請確認查詢條件或資料庫內容。


In [25]:
# === Agent 使用示範 ===
#question = "查詢與電影有關的文件"
question = "查詢與228 事件相關的文件"
response = agent.run(question)
if not response:
    print("⚠️ 無查詢結果，請檢查 SPARQL 或語句。")
else:
    print("🔎 查詢回覆：", response)

#print("SPARQL 查詢語句：", last_sparql_query)



> Entering new AgentExecutor chain...
Thought: 我需要查詢與228事件相關的文件，因此我可以使用SPARQL語言查詢相關的資訊。

Action: query_kg_tool
Action Input: PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s ?p ?o . FILTER (regex(str(?s), "228事件")) }
大型語言產生的 SPARQL 查詢語句：
 PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s ?p ?o . FILTER (regex(str(?s), "228事件")) }


Observation: {'s': 'http://example.org/doc/228事件(20).json', 'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'o': 'http://xmlns.com/foaf/0.1/Document'}
{'s': 'http://example.org/doc/您今天搭幾路？光復初期的基隆市公車(50)#event_228事件', 'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'o': 'http://example.org/schema#Event'}
{'s': 'http://example.org/doc/戰時相戀、和平分離─海角七號的故事背景(25)#event_228事件', 'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'o': 'http://example.org/schema#Event'}
{'

##### networkx 視覺化

In [26]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser'

def extract_label(uri):
    if uri.startswith("http"):
        return uri.rsplit("/", 1)[-1]
    return uri

sparql = SPARQLWrapper("http://192.168.133.39:8890/sparql")
#sparql.setQuery("""
#PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s ?p ?o . FILTER (regex(str(?s), "228事件")) }
#""")

sparql.setQuery(last_sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
#results 

# 建立 NetworkX 圖形
G = nx.DiGraph()

for result in results["results"]["bindings"]:
    s = extract_label(result["s"]["value"])
    p = extract_label(result["p"]["value"])
    if result["o"]["type"] == "uri":
        o = extract_label(result["o"]["value"])
    else:
        o = result["o"]["value"]
    G.add_edge(s, o, label=p)

pos = nx.spring_layout(G, seed=42)
edge_x, edge_y = [], []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=1, color='#888'),
                        hoverinfo='none', mode='lines')

node_x, node_y, node_text = [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

node_trace = go.Scatter(x=node_x, y=node_y, mode='markers+text', hoverinfo='text',
                        marker=dict(size=10, color='blue'), text=node_text)

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(title='SPARQL Knowledge Graph', showlegend=False))

fig.show()


##### cytoscape 視覺化

In [28]:
import dash
from dash import dcc, html, Input, Output, State
import dash_cytoscape as cyto
import requests
import threading
import webbrowser
import re

app = dash.Dash(__name__)

SPARQL_ENDPOINT = "http://192.168.133.39:8890/sparql"

def simplify_uri(uri):
    if '#' in uri:
        return uri.split('#')[-1]
    else:
        return uri.rstrip('/').split('/')[-1]

app.layout = html.Div([
    html.H2("SPARQL 知識圖譜查詢與視覺化"),

    dcc.Input(
        id='sparql-input',
        type='text',
        placeholder='輸入 SPARQL 查詢，例如：SELECT * WHERE { ?s ?p ?o } LIMIT 10',
        style={'width': '80%', 'margin-bottom': '10px'}
    ),
    html.Button('查詢', id='query-button', n_clicks=0, style={'margin-bottom': '20px'}),

    cyto.Cytoscape(
        id='cytoscape-graph',
        layout={'name': 'cose'},  # 或 'breadthfirst'
        style={'width': '100%', 'height': '600px'},
        elements=[],
        stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)',
                    'background-color': '#0074D9',
                    'color': 'white',
                    'text-valign': 'center',
                    'text-halign': 'center',
                    'font-size': 10,
                    'width': 'label',
                    'padding': '8px',
                    'shape': 'roundrectangle',
                    'border-width': 1,            # 邊框調整，預設是 0
                    'border-color': '#333333'   # 邊框顏色
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'curve-style': 'bezier',
                    'target-arrow-shape': 'triangle-backcurve',
                    'source-arrow-shape': 'none',
                    'target-arrow-color': '#000',
                    'line-color': '#000',
                    'arrow-scale': 1.5,
                    'width': 2,
                    'label': 'data(label)',        
                    # 加框設定 
                    'text-background-color': '#EEE',           # 背景顏色
                    'text-background-opacity': 1,              # 背景不透明度（0～1）
                    'text-background-shape': 'roundrectangle', # 形狀： rectangle / roundrectangle
                    'text-border-color': '#333',               # 框線顏色
                    'text-border-width': 1,                    # 框線寬度
                    'text-border-opacity': 1,                  # 框線不透明度（0～1）
                    # 其他文字樣式
                    'font-size':  9,
                    'text-rotation': 'autorotate',
                    'color': '#000'
                }
            }
        ]
    )
])

@app.callback(
    Output('cytoscape-graph', 'elements'),
    Input('query-button', 'n_clicks'),
    State('sparql-input', 'value')
)
def update_graph(n_clicks, sparql_query):
    if not sparql_query:
        return []

    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.get(SPARQL_ENDPOINT, params={'query': sparql_query}, headers=headers, timeout=10)
        data = response.json()
        bindings = data.get("results", {}).get("bindings", [])
    except Exception as e:
        print(f"查詢失敗: {e}")
        return []

    nodes = {}
    edges = []
    for result in bindings:
        s = result.get("s", {}).get("value", "")
        p = result.get("p", {}).get("value", "")
        o = result.get("o", {}).get("value", "")

        if s and s not in nodes:
            nodes[s] = {'data': {'id': s, 'label': simplify_uri(s)}}
        if o and o not in nodes:
            nodes[o] = {'data': {'id': o, 'label': simplify_uri(o)}}

        if s and o and p:
            edges.append({
                'data': {'source': s, 'target': o, 'label': simplify_uri(p)}
            })

    return list(nodes.values()) + edges

# 啟動 Dash 應用
def run_dash():
    webbrowser.open_new("http://127.0.0.1:8050/")
    app.run(debug=False, use_reloader=False)

threading.Thread(target=run_dash).start()
